In [1]:
from scvi.dataset import EbiData, MouseAtlas, UnionDataset, AnnDatasetFromAnnData
from Eval_basis import *
import scanpy as sc
import pandas as pd
import scipy.sparse as sparse
from tqdm import tqdm_notebook as tqdm
import urllib

[2019-09-04 17:24:09,154] INFO - scvi._settings | Added StreamHandler with custom formatter to 'scvi' logger.
/home/michael/anaconda3/envs/scvi/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [2]:
fpaths_and_fnames = {'data': "./data/mouse_atlas/cleaned_data_sparse.npz",
                     'cell': "./data/mouse_atlas/cell_annotation.csv",
                     'gene': "./data/mouse_atlas/gene_annotation.csv",
                     'pheno': "./data/mouse_atlas/phenotype_data.csv"}
mouse_union = UnionDataset("./data", 
                          gene_map_load_filename="gene_maps/ensembl_mouse_genes-proteincoding", 
                          low_memory=False)
mouse_union.join_datasets(data_source="memory", 
                          data_target="memory",
                          gene_datasets=[MouseAtlas("./data/mouse_atlas",
                                                    fpaths_and_fnames,
                                                    True,
                                                    False)])
mouse_union.name = "Mouse Atlas"

Concatenating datasets: 100%|██████████| 1/1 [15:49<00:00, 949.27s/it]
[2019-09-04 17:41:17,833] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.
[2019-09-04 17:41:29,668] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 17:41:29,705] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.
[2019-09-04 17:41:29,726] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 17:41:29,893] INFO - scvi.dataset.dataset | Joined 1 datasets to one of shape 1331984 x 22250.


In [3]:
conv = pd.read_csv("./data/gene_maps/hugo_mouse_genes-proteincoding.csv", header=0, index_col=0)
conv.index = conv.index.str.lower()

data_path = os.path.join(("./data"))
mouse_data_path = os.path.join(data_path, "mouse_data")
dsets = [] 
for file in os.listdir(f"{data_path}/mouse_data"):
#     if "droplet" in file:
    dset = sc.read_h5ad(os.path.join(mouse_data_path, file))
    dset.obs.rename(columns={"cell_ontology_class": "cell_types"}, inplace=True)
    
    dset = AnnDatasetFromAnnData(dset)
    
    gns_conved = conv.reindex(np.char.lower(dset.gene_names))["ensembl"]
    if not isinstance(dset.X, np.ndarray):
        X = dset.X.toarray()
    else:
        X = dset.X
    mask = ~gns_conved.isnull()
    
    dset.gene_names = gns_conved[mask].values.astype(str)
    dset.X = X[:, mask]
    dset.cell_types = np.array([ct.replace("ï", "i") for ct in dset.cell_types])
    
    dsets.append(dset)

[2019-09-04 17:46:05,741] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-09-04 17:46:06,310] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 17:46:06,311] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 17:46:06,500] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 17:46:06,696] INFO - scvi.dataset.dataset | Downsampled from 7916 to 7916 cells
[2019-09-04 17:46:07,715] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 17:46:08,866] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-09-04 17:46:09,170] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 17:46:09,171] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 17:46:09,252] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 17:46:09,338] IN

[2019-09-04 17:47:16,338] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-09-04 17:47:16,563] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 17:47:16,564] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 17:47:16,607] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 17:47:16,654] INFO - scvi.dataset.dataset | Downsampled from 1561 to 1561 cells
[2019-09-04 17:47:16,838] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 17:47:17,583] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-09-04 17:47:17,887] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 17:47:17,888] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 17:47:17,955] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 17:47:18,027] IN

[2019-09-04 17:47:53,970] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 17:47:54,212] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 17:47:54,456] INFO - scvi.dataset.dataset | Downsampled from 9734 to 9734 cells
[2019-09-04 17:47:55,859] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 17:47:59,551] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 17:47:59,553] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 17:47:59,958] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 17:48:00,370] INFO - scvi.dataset.dataset | Downsampled from 29322 to 29322 cells
[2019-09-04 17:48:05,984] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 17:48:10,673] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-09-04 17:48:11,117] INFO - scvi.data

In [4]:
mouse_muris_senis = UnionDataset("./data", 
                                 gene_map_load_filename="gene_maps/ensembl_mouse_genes-proteincoding", 
                                 low_memory=False)

In [5]:
mouse_muris_senis.join_datasets(data_source="memory", 
                                data_target="memory",
                                gene_datasets=dsets)
mouse_muris_senis.name = "Tabula Muris Senis"

Concatenating datasets: 100%|██████████| 38/38 [04:59<00:00,  6.49s/it]
[2019-09-04 17:53:25,609] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.
[2019-09-04 17:53:31,671] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 17:53:31,677] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.
[2019-09-04 17:53:31,680] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 17:53:31,710] INFO - scvi.dataset.dataset | Joined 38 datasets to one of shape 290905 x 22250.


In [6]:
dsets = None
mouse_muris_senis.cell_types = np.array([ct.replace("ï", "i") for ct in mouse_muris_senis.cell_types])

In [7]:
ebi_datasets_ids = ['E-MTAB-6946', 'E-MTAB-7320',
                    "E-MTAB-7417", "E-MTAB-6173", "E-GEOD-81682", 
                    "E-ENAD-13", "E-GEOD-99235", "E-GEOD-71585", 
                    "E-GEOD-87631", "E-GEOD-90848", "E-MTAB-4547",
                   "E-MTAB-6970", "E-MTAB-5802", "E-MTAB-5661", 
                    "E-GEOD-99058", "E-GEOD-106973", "E-MTAB-7365",
                   "E-MTAB-5553", "E-MTAB-6925", "E-MTAB-6976"]

ebi_data = []
for e_id in tqdm(ebi_datasets_ids, desc="Loading EBI datasets"):
    try:
        data = EbiData("./data", e_id)
    except urllib.error.HTTPError:
        data = EbiData("./data", e_id, result_file='raw')
    data.X = sparse.csr_matrix(data.X)
    ebi_data.append(data)

/home/michael/anaconda3/envs/scvi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,20) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
[2019-09-04 17:54:43,219] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 17:54:43,223] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 17:54:46,383] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 17:54:49,442] INFO - scvi.dataset.dataset | Downsampled from 167276 to 167276 cells
[2019-09-04 17:54:49,536] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 17:55:22,947] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 17:55:22,950] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 17:55:24,050] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-

[2019-09-04 17:56:00,425] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 17:56:00,426] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 17:56:00,430] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 17:56:00,435] INFO - scvi.dataset.dataset | Downsampled from 250 to 250 cells
[2019-09-04 17:56:00,500] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 17:56:00,830] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-09-04 17:56:00,846] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 17:56:00,846] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 17:56:00,849] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 17:56:00,851] INFO - scvi.dataset.dataset | Downsampled from 141 to 141 cells
[2019-09-04 17:56:00,891] INFO - scvi.dataset.dataset | Com

In [8]:
complete_mouse = UnionDataset("./data", 
                              gene_map_load_filename="gene_maps/ensembl_mouse_genes-proteincoding", 
                              low_memory=False)

In [9]:
complete_mouse.join_datasets(data_source="memory", 
                             data_target="loom", 
                             out_filename="mouse_data_all.loom", 
                             gene_datasets=ebi_data + [mouse_muris_senis, mouse_union])
ebi_data = None
mouse_muris_senis = None
mouse_atlas = None

Writing dataset 'UnionDataset - Mouse Atlas' to file: 100%|██████████| 22/22 [28:19<00:00, 387.90s/it]      
[2019-09-04 18:24:21,874] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.


In [3]:
complete_mouse.X

<1868228x22250 sparse matrix of type '<class 'numpy.float64'>'
	with 1878290427 stored elements in Compressed Sparse Row format>

In [2]:
complete_mouse = UnionDataset("./data", 
                              gene_map_load_filename="gene_maps/ensembl_mouse_genes-proteincoding", 
                              data_load_filename="mouse_data_all.loom",
                              low_memory=False)

Loading from file to memory iteratively: 100%|██████████| 374/374 [20:28<00:00,  3.98s/it]
[2019-09-03 15:32:25,993] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.
[2019-09-03 15:32:46,109] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-03 15:32:46,368] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-03 15:32:46,412] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.


In [ ]:
n_epochs = 100
colors=None

print("Training VAE")

trainer = train_vae(complete_mouse, "./data", f"../trained_models/mouse_data_full_model", n_epochs=n_epochs)
# trainer_small = train_vae(data_small, "./data", f"small_{tissue}_data_portion", n_epochs=n_epochs)



Training VAE
Initializing training.



training:   4%|▍         | 4/100 [15:52<6:20:52, 238.05s/it]

training:   9%|▉         | 9/100 [41:09<6:37:03, 261.80s/it]

training:  14%|█▍        | 14/100 [1:06:25<6:20:49, 265.69s/it]

training:  16%|█▌        | 16/100 [1:19:57<7:31:18, 322.36s/it]

In [ ]:
mouse_ebi_celltype_data = UnionDataset("./data", 
                              gene_map_load_filename="gene_maps/ensembl_mouse_genes-proteincoding", 
                              low_memory=False)
mouse_ebi_celltype_data.join_datasets(data_source="memory", 
                             data_target="memory", 
                             gene_datasets=[EbiData("./data")])

In [ ]:
dot_size = (mpl.rcParams['lines.markersize'] ** 2.0)

posterior_big = plot_tsne(trainer, trainer.model, complete_mouse, f"./maxdata_model", image_datatype="pdf",
                          colors=colors, s=dot_size, edgecolors='black')

# posterior_ebi_annotated = trainer.create_posterior(model, ebi_with_celltypes, indices=np.arange(len(dataset)))
posterior_ebi_annotated = plot_tsne(trainer_big, trainer_big.model, ebi_with_celltypes, f"./plots/ebi_annotated_in_maxmodel.pdf",
                                    image_datatype="pdf", colors=colors, s=dot_size, edgecolors='black')